In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
def pca_features(indep_X, n):
    pca = PCA(n_components=n)
    pca_features = pca.fit_transform(indep_X)
    return pca_features, pca
    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2
 
def Linear(X_train,y_train,X_test,y_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2   
    
def svm_linear(X_train,y_train,X_test,y_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
    
def svm_NL(X_train,y_train,X_test,y_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def Decision(X_train,y_train,X_test,y_test):
        
      
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def random(X_train,y_train,X_test,y_test):       
        
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 
    
    
def PCA_regression(acclin, accsvml, accsvmnl, accdes, accrf, index_names):
    dataframe = pd.DataFrame(index=index_names,
                             columns=['Linear', 'SVMl', 'SVMnl', 'Decision', 'Random'])
    for number, idex in enumerate(dataframe.index):
        dataframe['Linear'][idex] = acclin[number]
        dataframe['SVMl'][idex] = accsvml[number]
        dataframe['SVMnl'][idex] = accsvmnl[number]
        dataframe['Decision'][idex] = accdes[number]
        dataframe['Random'][idex] = accrf[number]
    return dataframe

In [3]:
dataset1=pd.read_csv("prep.csv",index_col=None)

df2=dataset1

df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']

In [4]:
kbest = [2, 3, 5, 10] 

acclin=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]
index_names=[]

In [5]:
for k in kbest:
    pca_data, pca = pca_features(indep_X, k)
    X_train, X_test, y_train, y_test = split_scalar(pca_data, dep_Y)

    r2_lin = Linear(X_train, y_train, X_test, y_test)
    acclin.append(r2_lin)

    r2_svm_l = svm_linear(X_train, y_train, X_test, y_test)
    accsvml.append(r2_svm_l)

    r2_svm_nl = svm_NL(X_train, y_train, X_test, y_test)
    accsvmnl.append(r2_svm_nl)

    r2_d = Decision(X_train, y_train, X_test, y_test)
    accdes.append(r2_d)

    r2_r = random(X_train, y_train, X_test, y_test)
    accrf.append(r2_r)

    index_names.append(f'PCA_{k}')

In [6]:
result = PCA_regression(acclin, accsvml, accsvmnl, accdes, accrf, index_names)

In [7]:
result

,Linear,SVMl,SVMnl,Decision,Random
PCA_2,0.236027,-0.055249,0.32131,0.175347,0.398003
PCA_3,0.296502,0.238085,0.326956,0.348958,0.528212
PCA_5,0.299463,0.181199,0.556325,0.479167,0.604167
PCA_10,0.563064,0.440839,0.837543,0.479167,0.803385
